In [17]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
from numpy.linalg import solve,norm
# from robot_descriptions.loaders.pinocchio import load_robot_description
# import gepetto.corbaserver

In [18]:
urdf_filename = "/home/adi/hum_rob_ws/src/six_dof/urdf/6dof_fixed.urdf"
mesh_dir = "/home/adi/hum_rob_ws/src/six_dof/meshes"

ros_package_path = "/home/adi/hum_rob_ws/src"  # Replace with your path
os.environ["ROS_PACKAGE_PATH"] = ros_package_path

In [19]:
# building model from URDF, model is an object that includes kinematic and ineratia params
model = pin.buildModelFromUrdf(urdf_filename)

visual_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.VISUAL, package_dirs=mesh_dir
)  # very important

collision_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.COLLISION, package_dirs=mesh_dir
)

geom_data = pin.GeometryData(collision_model)


# data is an object that holds values that are a result of computation
data = model.createData()

# print('standard model: dim=' + str(len(model.joints)))
# for jn in model.joints:
#     print(jn)
# print('-' * 30)


# # Create a list of joints to lock
# jointsToLock = ['right_foot_shin', 'right_shin_thigh', 'right_thigh_body']
 
# # Get the ID of all existing joints
# jointsToLockIDs = []
# for jn in jointsToLock:
#     if model.existJointName(jn):
#         jointsToLockIDs.append(model.getJointId(jn))
#     else:
#         print('Warning: joint ' + str(jn) + ' does not belong to the model!')

# print(jointsToLockIDs)
 
# geom_models = [visual_model, collision_model]
# model_reduced, geometric_models_reduced = pin.buildReducedModel(
#     model,
#     list_of_geom_models=geom_models,
#     list_of_joints_to_lock=jointsToLockIDs,
#     reference_configuration=pin.neutral(model))
# # geometric_models_reduced is a list, ordered as the passed variable "geom_models" so:
# visual_model_reduced, collision_model_reduced = geometric_models_reduced[
#     0], geometric_models_reduced[1]

In [20]:
visualizer = MeshcatVisualizer(model, collision_model, visual_model)


visualizer.initViewer()
visualizer.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7026/static/


In [21]:
q = pin.neutral(model)
print(q)
eps = 1e-3
DT = 1e-1
damp = 1e-1
IT_MAX=10000
JOINT_ID = 10

oMdes = pin.SE3(np.eye(3), np.array([0.092, 0.02, 0.05])) #think of oMdes as oTdes, the transfromation from desired to origin expressed in origin
i = 0



[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [22]:
visualizer.display(q)
visualizer.viewer.jupyter_cell()

In [23]:
while True:
    pin.forwardKinematics(model, data, q)
    
    iMd = data.oMi[JOINT_ID].actInv(oMdes) #active inverse, chnages basis and moves the vector/point to the appropriate position in the new basis
    err = pin.log(iMd).vector  # in joint frame  The logarithm map (log()) is a mathematical operation that maps elements from the group SE(3) to its associated Lie algebra, se(3). This operation effectively converts the complex rotational and translational transformation into a simpler vector representation that captures the "difference" or "error" between two poses in SE(3).
    norm_err=norm(err)
    if norm_err < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pin.computeJointJacobian(
        model, data, q, JOINT_ID
    )  # jacobian computed in e-e frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    q_dot = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pin.integrate(model, q, q_dot * DT)
    
    # #Collision check
    # pin.updateGeometryPlacements(
    #     model, data, collision_model, geom_data, q
    # )  # updates geom_data by reference

    # # an active pair is a pair of bodies in a joint considered for collision
    # collision_detected = pin.computeCollisions(
    #     model, data, collision_model, geom_data, q, True
    # )  # this returns data,geom_data. polymorphic function

    # if not collision_detected:
    #     for idx in range(model.nq):
    #         # Assuming model.lowerPositionLimit and model.upperPositionLimit store the limits
    #         q[idx] = max(
    #             model.lowerPositionLimit[idx],
    #             min(q[idx], model.upperPositionLimit[idx]),
    #         )
    
    
    visualizer.display(q)
    print(norm_err)
    #print(i)
    i+=1
    #time.sleep(0.05)

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)



0.05395811411542384
0.05345138290444526
0.05296574188837171
0.052499441062665464
0.052050917688289164
0.051618771750342775
0.0512017450647324
0.05079870346848584
0.0504086216142647
0.050030569963004756
0.04966370363163905
0.04930725280668785
0.04896051448030724
0.048622845304196693
0.04829365538956291
0.047972402908973324
0.04765858937916518
0.04735175552337251
0.04705147762807198
0.04675736432271488
0.04646905372243885
0.04618621088329467
0.04590852552749447
0.04563571000282875
0.0453674974459543
0.04510364012387899
0.04484390793184122
0.04458808702900593
0.04433597859610668
0.04408739770141703
0.043842172263339786
0.04360014209948734
0.04336115805347242
0.0431250811917586
0.042891782063875644
0.04266114002011761
0.04243304258153481
0.0422073848576157
0.04198406900756939
0.04176300374154667
0.04154410385852251
0.041327289817883396
0.04111248734205337
0.04089962704774053
0.04068864410360719
0.04047947791235943
0.04027207181542961
0.04006637281857161
0.03986233133683687
0.03965990095751

In [24]:
pin.forwardKinematics(model,data,q)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model.names, data.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))

universe                 :  0.000  0.000  0.000
right_foot_shin          :  0.004  0.000  0.000
right_shin_thigh         :  0.017 -0.049  0.120
right_thigh_body         : -0.025 -0.131  0.258
right_thigh_body_lateral : -0.025 -0.131  0.258
body_left_thigh_lateral  :  0.071 -0.151  0.280
left_thigh_shin          :  0.066 -0.041  0.164
left_shin_foot           :  0.088  0.020  0.049
body_left_thigh_lateral  :  0.071 -0.151  0.280
left_thigh_shin          :  0.066 -0.041  0.164
left_shin_foot           :  0.088  0.020  0.049
